<a href="https://colab.research.google.com/github/tanvir-ahammedd/llm-finetuning-lab/blob/main/Gemma_9b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q
!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes xformers datasets -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 128.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 124.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.5 MB/

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-9b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.3: Fast Gemma2 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

In [ ]:
FastLanguageModel.for_inference(model)

alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

instruction = "You are a helpful assistant who can answer questions"
input_text = "Who developed GPT models?"

inputs = tokenizer([alpaca_prompt.format(instruction, input_text)], return_tensors='pt').to('cuda')
outputs = model.generate(**inputs, max_new_tokens=100)
response = tokenizer.batch_decode(outputs)[0]
print("\n Pre-trained model output:\n", response)


 Pre-trained model output:
 <bos>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
You are a helpful assistant who can answer questions

### Response:
Who developed GPT models?<eos>


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    max_seq_length = max_seq_length,
)

Unsloth 2025.10.3 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


In [ ]:
from google.colab import files
uploaded = files.upload()

from datasets import load_dataset

dataset = load_dataset("csv", data_files="my_dataset.csv")["train"]

print("Example raw row:")
print(dataset[0])

Saving my_dataset.csv to my_dataset (1).csv
Example raw row:
{'id': 1, 'instruction': 'স্ট্রিং থেকে প্রদত্ত অক্ষরের প্রথম এবং শেষ উপস্থিতি অপসারণের জন্য একটি পাইথন ফাংশন লিখুন।', 'response': 'def remove_Occ(s,ch): \r\n    for i in range(len(s)): \r\n        if (s[i] == ch): \r\n            s = s[0 : i] + s[i + 1:] \r\n            break\r\n    for i in range(len(s) - 1,-1,-1):  \r\n        if (s[i] == ch): \r\n            s = s[0 : i] + s[i + 1:] \r\n            break\r\n    return s ', 'test_list': '"[\'assert remove_Occ(\\"hello\\",\\"l\\") == \\"heo\\"\', \'assert remove_Occ(\\"abcda\\",\\"a\\") == \\"bcd\\"\', \'assert remove_Occ(\\"PHP\\",\\"P\\") == \\"H\\"\']"'}


In [ ]:
alpaca_prompt = """Below is a programming instruction with sample test cases. Write the correct Python function to complete the request.

### Instruction:
{}

### Example Tests:
{}

### Response:
{}"""

def format_input_prompt(examples):
    instructions = examples["instruction"]
    test_lists = examples["test_list"]
    outputs = examples["response"]

    texts = []
    for instruction, test_list, output in zip(instructions, test_lists, outputs):
        text = alpaca_prompt.format(instruction, test_list, output) + tokenizer.eos_token
        texts.append(text)

    return {"text": texts}

dataset = dataset.map(format_input_prompt, batched=True)

print("\n Example formatted text:\n")
print(dataset["text"][0])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]


 Example formatted text:

Below is a programming instruction with sample test cases. Write the correct Python function to complete the request.

### Instruction:
স্ট্রিং থেকে প্রদত্ত অক্ষরের প্রথম এবং শেষ উপস্থিতি অপসারণের জন্য একটি পাইথন ফাংশন লিখুন।

### Example Tests:
"['assert remove_Occ(\"hello\",\"l\") == \"heo\"', 'assert remove_Occ(\"abcda\",\"a\") == \"bcd\"', 'assert remove_Occ(\"PHP\",\"P\") == \"H\"']"

### Response:
def remove_Occ(s,ch): 
    for i in range(len(s)): 
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    for i in range(len(s) - 1,-1,-1):  
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    return s <eos>


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        num_train_epochs = 1,
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1234,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Trainer_Stats = trainer.train()
Trainer_Stats

Map (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 54,018,048 of 9,295,724,032 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.696600
2,1.981500
3,1.909100
4,1.886200
5,1.560100
6,1.322000
7,1.248200
8,0.925600
9,0.814100
10,0.821500


TrainOutput(global_step=30, training_loss=0.8765415916840236, metrics={'train_runtime': 373.8931, 'train_samples_per_second': 0.642, 'train_steps_per_second': 0.08, 'total_flos': 3046572150236160.0, 'train_loss': 0.8765415916840236, 'epoch': 0.48})

In [ ]:
model.save_pretrained("./best_model")
tokenizer.save_pretrained("./best_model")

('./best_model/tokenizer_config.json',
 './best_model/special_tokens_map.json',
 './best_model/tokenizer.model',
 './best_model/added_tokens.json',
 './best_model/tokenizer.json')

In [ ]:
FastLanguageModel.for_inference(model)

test_instruction = "একটি পাইথন ফাংশন লিখুন যা একটি তালিকার সব সংখ্যার গড় বের করবে।"
test_input = "[]"

inputs = tokenizer([alpaca_prompt.format(test_instruction, test_input, "")], return_tensors='pt').to('cuda')
outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.7)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print("\n Fine-tuned model output:\n")
print(response)


 Fine-tuned model output:

Below is a programming instruction with sample test cases. Write the correct Python function to complete the request.

### Instruction:
একটি পাইথন ফাংশন লিখুন যা একটি তালিকার সব সংখ্যার গড় বের করবে।

### Example Tests:
[]

### Response:
def average(list1):
    sum = 0
    for i in range(len(list1)):
        sum += list1[i]
    return sum / len(list1)


In [ ]:
FastLanguageModel.for_inference(model)

test_instruction = "একটি পাইথন ফাংশন লিখুন যা এক থেকে একশত পর্যন্ত গণনা করবে।"
test_input = "[]"

inputs = tokenizer([alpaca_prompt.format(test_instruction, test_input, "")], return_tensors='pt').to('cuda')
outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.7)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print("\n Fine-tuned model output:\n")
print(response)


 Fine-tuned model output:

Below is a programming instruction with sample test cases. Write the correct Python function to complete the request.

### Instruction:
একটি পাইথন ফাংশন লিখুন যা এক থেকে একশত পর্যন্ত গণনা করবে।

### Example Tests:
[]

### Response:
def count_from_one_to_one_hundred():
  for i in range(1, 101):
    print(i)


In [ ]:
FastLanguageModel.for_inference(model)

test_instruction = "একটি নির্দিষ্ট সংখ্যার পূর্ববর্তী প্যালিনড্রোম খুঁজে পেতে একটি ফাংশন লিখুন।"
test_input = "[]"

inputs = tokenizer([alpaca_prompt.format(test_instruction, test_input, "")], return_tensors='pt').to('cuda')
outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.7)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print("\n Fine-tuned model output:\n")
print(response)


 Fine-tuned model output:

Below is a programming instruction with sample test cases. Write the correct Python function to complete the request.

### Instruction:
একটি নির্দিষ্ট সংখ্যার পূর্ববর্তী প্যালিনড্রোম খুঁজে পেতে একটি ফাংশন লিখুন।

### Example Tests:
[]

### Response:
def previous_palindrome(n):
  while True:
    n -= 1
    if str(n) == str(n)[::-1]:
      return n


In [ ]:
FastLanguageModel.for_inference(model)

test_instruction = "হিপ ক্যু অ্যালগরিদম ব্যবহার করে পূর্ণসংখ্যার একটি অ্যারের তিনটি সংখ্যার সর্বোচ্চ উৎপাদনের জন্য একটি ফাংশন লিখুন।"
test_input = "[]"

inputs = tokenizer([alpaca_prompt.format(test_instruction, test_input, "")], return_tensors='pt').to('cuda')
outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.7)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print("\n Fine-tuned model output:\n")
print(response)


 Fine-tuned model output:

Below is a programming instruction with sample test cases. Write the correct Python function to complete the request.

### Instruction:
হিপ ক্যু অ্যালগরিদম ব্যবহার করে পূর্ণসংখ্যার একটি অ্যারের তিনটি সংখ্যার সর্বোচ্চ উৎপাদনের জন্য একটি ফাংশন লিখুন।

### Example Tests:
[]

### Response:
def max_product_of_three(arr):
    arr.sort()
    return max(arr[0] * arr[1] * arr[2], arr[-1] * arr[-2] * arr[-3])


In [ ]:
FastLanguageModel.for_inference(model)

test_instruction = "একটি ফাংশন লিখুন যাতে সবচেয়ে ছোট স্ট্রিং এর দৈর্ঘ্য খুঁজে পাওয়া যায় যার মধ্যে str1 এবং str2 উভয়ই সাবসেকভেন্স হিসাবে থাকে।"
test_input = "[]"

inputs = tokenizer([alpaca_prompt.format(test_instruction, test_input, "")], return_tensors='pt').to('cuda')
outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.7)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print("\n Fine-tuned model output:\n")
print(response)


 Fine-tuned model output:

Below is a programming instruction with sample test cases. Write the correct Python function to complete the request.

### Instruction:
একটি ফাংশন লিখুন যাতে সবচেয়ে ছোট স্ট্রিং এর দৈর্ঘ্য খুঁজে পাওয়া যায় যার মধ্যে str1 এবং str2 উভয়ই সাবসেকভেন্স হিসাবে থাকে।

### Example Tests:
[]

### Response:
def find_min_length(str1, str2):
    n1 = len(str1)
    n2 = len(str2)
    dp = [[0 for i in range(n2 + 1)] for j in range(n1 + 1)]
    for i in range(1, n1 + 1):
        for j in range(1, n2 + 1):
            if str1[i - 1] == str2[j - 1]:
                dp[i][j] = 1 + dp[i - 1][j - 1]
            else:
                dp[i][j] = max(
